#####Libraries
***

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
!pip install yfinance --upgrade --no-cache-dir
import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()
# pip install TA-lib
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz

!tar -xzf ta-lib-0.4.0-src.tar.gz
%cd ta-lib/
!./configure --prefix=$HOME
!make
!make install

!TA_LIBRARY_PATH=~/lib TA_INCLUDE_PATH=~/include pip install ta-lib
import talib as ta
import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 6.8 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
--2022-08-31 09:50:14--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 204.68.111.105
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|204.68.111.105|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2022-08-31 09:50:14--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 204.68.111.105
Reusing existing connection to prdownlo

#####Load Dataset
***

In [4]:
#φτιάχνω τη λίστα με τα assets μου -> 50 μετοχές 
#διόρθωση : θα βγάλω τυχαία άλλες 4 μετοχές μαζί με την OGN επειδή δεν έχει το ίδιο μέγεθος με τις άλλες 
#βγάζω OGN ,WRK,HST,CE,ACN
assets= ["ABBV","AEP","AIZ","ALLE","AMAT","AMP","AMZN","AVB","AVY",   
  "AXP","BDX","BF-B","BMY","BR","CARR","CDW","CHTR","CNC",   
  "CNP","COP","CTAS","CZR","DG","DPZ","DRE","DXC","META","FTV",   
  "GOOG","GPC","HIG","JPM","KR","PG","PPL","PRU",   
  "PYPL","RE","ROL","ROST","UNH","URI","V","VRSK","XOM"]

In [5]:
#άρα πρέπει να κατεβάσουμε όλα τα adj close για όλες τις τιμές όλων των μετοχών και να κάνουμε απόλυτο ranking και σε 5 κλασεις (targets)
r=pd.DataFrame(index=pdr.get_data_yahoo(assets[0],'2021-01-01','2022-04-01')['Adj Close'].index)
#παίρνω τιμές για ένα χρόνο και 3 μήνες
for i in assets:
  # print('i metoxi me onoma:',i,'exei megethos',pdr.get_data_yahoo(i,'2021-01-01','2022-04-01')['Adj Close'].count())
  temp= pdr.get_data_yahoo(i,'2021-01-01','2022-04-01')['Adj Close']
  temp=temp.dropna()
  r[i]=temp.values
#φτιάχνω ένα αντίγραφο με τις τιμές για να το έχω 
rd=pd.DataFrame(r)
rd

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-04,97.798592,76.819260,128.241989,112.358444,85.672470,182.556015,159.331497,148.850800,148.753860,115.625526,...,70.458305,231.919998,218.876434,37.469650,115.092255,342.416077,229.649994,215.285767,200.928055,37.906906
2021-01-05,98.809891,76.715630,128.280807,112.574356,88.364830,181.488495,160.925507,148.420074,149.077026,116.242622,...,71.140755,234.910004,217.072540,38.086956,115.190552,337.811310,235.570007,212.072678,200.026810,39.733746
2021-01-06,97.956322,77.327995,133.911026,115.136009,89.577881,191.901703,156.919006,152.401672,154.776474,120.542824,...,75.355347,226.830002,229.939713,38.714066,120.224060,351.988037,256.549988,210.204147,198.135162,40.747643
2021-01-07,99.004723,75.264778,135.260315,118.168732,93.256462,194.221130,158.108002,150.324753,159.604355,119.589340,...,75.023346,235.039993,231.590103,39.360771,119.958611,357.631226,264.679993,211.380630,199.036392,41.067341
2021-01-08,99.524284,74.595879,134.551666,117.756523,94.242661,195.599213,159.134995,151.645554,153.503403,119.707306,...,74.423882,242.460007,229.296799,39.978081,120.342026,356.024506,260.149994,213.002014,198.967087,41.524048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-25,158.487930,95.333145,183.029373,111.845345,136.376923,311.729828,164.773499,243.735245,172.392029,189.033966,...,118.004997,113.760002,295.426666,33.621418,89.881897,511.206970,363.670013,217.628159,207.601166,83.524574
2022-03-28,159.116653,96.091171,182.225754,113.020569,136.973938,308.232391,168.990494,246.446274,172.750473,188.457764,...,117.223892,116.169998,294.201508,34.228012,91.216866,511.376343,363.399994,219.959564,211.194260,81.181572
2022-03-29,159.322952,97.154373,179.884308,114.584213,140.735077,309.342041,169.315002,254.599045,178.127136,189.689651,...,118.004997,121.180000,298.953888,34.953945,95.191925,508.915131,365.200012,227.282578,214.388123,80.750229


In [6]:
rd1= rd.pct_change().shift(-1)
rd2=rd1.rank(axis=1,method='first')
rd2=rd2.dropna()
rd2.astype(int)
jk=rd2.copy()
jk.astype(int)

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-04,34,14,15,21,41,9,33,12,22,25,...,32,36,6,38,17,5,40,3,11,44
2021-01-05,8,14,37,25,20,40,3,28,31,29,...,41,1,42,21,36,35,44,7,6,26
2021-01-06,25,2,23,36,44,28,20,3,40,8,...,9,43,19,30,11,29,41,18,16,21
2021-01-07,28,8,13,17,37,31,30,34,1,22,...,11,45,6,42,26,14,3,33,19,38
2021-01-08,39,23,22,29,42,26,5,20,43,21,...,27,7,16,28,10,15,13,11,9,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-24,21,16,27,4,3,28,22,40,45,24,...,38,2,37,25,5,8,36,19,7,42
2022-03-25,22,27,9,31,23,4,45,36,19,13,...,8,42,11,41,38,18,16,32,40,2
2022-03-28,7,20,2,24,36,11,9,41,39,14,...,15,43,26,33,44,5,12,42,25,4


In [7]:
def rank_data(df1):
  for i in range(len(df1.index)):
    d=df1.loc[df1.index[i]]  #της κάθε μέρας d[1] δίνει μόνο τις τιμές 
    #print(d)
    d=d.sort_values()
    d[0:9]=1
    d[9:18]=2
    d[18:27]=3
    d[27:36]=4
    d[36:45]=5
    df1.loc[df1.index[i]]=d
    #print(df1.loc[df1.index[i]])
    #print(d)
  df1=df1.astype(int)
  return df1
r5=rd2.copy()
rd3=rank_data(r5)
rd3

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-04,4,2,2,3,5,1,4,2,3,3,...,4,4,1,5,2,1,5,1,2,5
2021-01-05,1,2,5,3,3,5,1,4,4,4,...,5,1,5,3,4,4,5,1,1,3
2021-01-06,3,1,3,4,5,4,3,1,5,1,...,1,5,3,4,2,4,5,2,2,3
2021-01-07,4,1,2,2,5,4,4,4,1,3,...,2,5,1,5,3,2,1,4,3,5
2021-01-08,5,3,3,4,5,3,1,3,5,3,...,3,1,2,4,2,2,2,2,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-24,3,2,3,1,1,4,3,5,5,3,...,5,1,5,3,1,1,4,3,1,5
2022-03-25,3,3,1,4,3,1,5,4,3,2,...,1,5,2,5,5,2,2,4,5,1
2022-03-28,1,3,1,3,4,2,1,5,5,2,...,2,5,3,4,5,1,2,5,3,1


#####Necessary funtions for RPS calculation 
***

In [14]:
def RPS_calculation(hist_data, submission, asset_no=44):
    # print('check')
    if hist_data.shape[0]<=asset_no:
        return np.nan    
    # print('check')
    asset_id = pd.unique(hist_data.symbol)
    
    for i in range(len(pd.unique(hist_data.date))):
        if len(hist_data[hist_data.date == pd.unique(hist_data.date)[i]])<len(asset_id):
            for asset in [x for x in asset_id if x not in hist_data[hist_data.date == pd.unique(hist_data.date)[i]].symbol.values]:
                right_price = hist_data[hist_data.symbol==asset].sort_values(by='date')
                right_price = right_price[right_price.date <= pd.unique(hist_data.date)[i]]
                right_price = right_price.price.iloc[-1]
                hist_data = hist_data.append({'date' : pd.unique(hist_data.date)[i],
                                               'symbol' : asset,
                                               'price' : right_price}, ignore_index=True)

    #Compute percentage returns
    asset_id = sorted(asset_id) 

    #Compute percentage returns
    returns = pd.DataFrame(columns = ["ID", "Return"])
    
    min_date = min(hist_data.date)
    max_date = max(hist_data.date)
    
    for i in range(0,len(asset_id)):
        temp = hist_data.loc[hist_data.symbol==asset_id[i]]
        
        open_price = float(temp.loc[temp.date==min_date].price)
        close_price = float(temp.loc[temp.date==max_date].price)
        
        returns = returns.append({'ID': temp.symbol.iloc[0], 
                                'Return': (close_price - open_price)/open_price}, ignore_index=True)

    #Define the relevant position of each asset
    ranking = pd.DataFrame(columns=["ID", "Position", "Return"])
    ranking.ID = list(asset_id)
    ranking.Return = returns.Return
    ranking.Position = ranking.Return.rank(method = 'min')

    #Handle Ties
    Series_per_position = pd.DataFrame(columns=["Position","Series", "Rank", "Rank1", "Rank2","Rank3", "Rank4", "Rank5"])
    Series_per_position.Position = list(pd.unique(ranking.Position.sort_values(ascending=True)))
    temp = ranking.Position.value_counts()
    temp = pd.DataFrame(zip(temp.index, temp), columns = ["Rank", "Occurencies"])
    temp = temp.sort_values(by = ["Rank"],ascending=True)
    Series_per_position.Series = list(temp.Occurencies)
    Series_per_position

    total_ranks = Series_per_position.Position.values[-1]
    for i in range(0,Series_per_position.shape[0]):
    
        start_p = Series_per_position.Position[i]
        end_p = Series_per_position.Position[i] + Series_per_position.Series[i]
        temp = pd.DataFrame(columns = ["Position","Rank", "Rank1", "Rank2", "Rank3", "Rank4","Rank5"])
        temp.Position = list(range(int(start_p),int(end_p)))

        if(temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank=1)
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank1=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank=2)
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank2=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank=3)
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank3=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank=4)
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank4=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank=5)
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank5=1.0)
        temp = temp.fillna(0)
        Series_per_position.iloc[i,2:Series_per_position.shape[1]] = temp.mean(axis = 0).iloc[1:temp.shape[1]]

    Series_per_position = Series_per_position.drop('Series', axis = 1)
    ranking = pd.merge(ranking,Series_per_position, on = "Position")
    ranking = ranking[["ID", "Return", "Position", "Rank", "Rank1", "Rank2", "Rank3", "Rank4", "Rank5"]]
    ranking = ranking.sort_values(["Position"])
    # print(ranking)
    #Evaluate submission
    rps_sub = []
    #for aid in list((pd.unique(ranking.ID))):
    for aid in asset_id:

        target = np.cumsum(ranking.loc[ranking.ID==aid].iloc[:,4:9].values).tolist()
        frc = np.cumsum(submission.loc[submission.ID==aid].iloc[:,1:6].values).tolist()
        rps_sub.append(np.mean([(a - b)**2 for a, b in zip(target, frc)]))
    submission["RPS"] = rps_sub
    
    output = {'RPS' : np.mean(rps_sub),
              'details' : submission}
    
    return(output)

In [15]:
def create_historical(prob):
  prob=prob.rename_axis("symbol", axis="columns")
  prob=prob.T
  prob['date']=prob.columns.date[0]
  prob.columns = ['price', 'date']
  prob=prob.reset_index()
  prob=prob[['symbol','date','price']]
  return prob

#####Preparation of the Data-**Features**
***

In [10]:
stock=dict()
for i in assets:
  #παίρνω δεδομένα για 15 μήνες περιπου->από Ιανουάριο 2021 μέχρι Μάρτιο 2022
  stock[i]=pdr.get_data_yahoo(i,'2021-01-01','2022-04-01') 
  # print(df)
  #από αυτές κρατάμε μόνο τις πρώτες 5 στήλες 
  stock[i]=stock[i].iloc[:,:5]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [11]:
stock

{'ABBV':                   Open        High         Low       Close   Adj Close
 Date                                                                  
 2021-01-04  107.180000  107.349998  103.860001  105.410004   97.798584
 2021-01-05  105.410004  107.019997  104.629997  106.500000   98.809883
 2021-01-06  104.750000  107.190002  104.180000  105.580002   97.956322
 2021-01-07  106.110001  107.059998  105.570000  106.709999   99.004715
 2021-01-08  106.839996  107.529999  105.760002  107.269997   99.524284
 ...                ...         ...         ...         ...         ...
 2022-03-25  160.550003  161.979996  160.229996  161.330002  158.487930
 2022-03-28  160.610001  162.029999  159.529999  161.970001  159.116653
 2022-03-29  162.990005  163.279999  161.289993  162.179993  159.322952
 2022-03-30  161.720001  164.350006  161.720001  163.750000  160.865295
 2022-03-31  162.929993  164.660004  162.100006  162.110001  159.254181
 
 [314 rows x 5 columns],
 'AEP':                  Open

In [12]:
#---Prepare the new features that have been selected previously---#

for j in assets:
    df=stock[j] #all the days we need so we can fix the columns 
    # print('to df ftanei mexri ',stock[j].iloc[:-61].tail(1))
    # print(df)
    #τώρα θα προσθέσουμε τις καινούριες στήλες με τα features μας 
    df['(high2-high1)/high2']= (df['High'].shift(2)-df['High'].shift(1))/df['High'].shift(2)
    df['(low2-low1)/low2']= (df['Low'].shift(2)-df['Low'].shift(1))/df['Low'].shift(2)
    df['av_3']=df['Adj Close'].shift(1).rolling(window=3).mean().pct_change()
    df['av_10']=df['Adj Close'].shift(1).rolling(window=10).mean().pct_change()
    df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    df['ADX'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=6)/100
    df['ADX_14'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=14)/100
    df['WILLR']= ta.WILLR(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1),timeperiod=14)/100
    # df['dist_from_mean_10'] = (df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=10).mean()))
    df = df.iloc[1: , :]
    df['absolute rank']= jk[j].astype(int).shift(1)
    df['Rank']= rd3[j]
    stock[j]=df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [13]:
stock

{'ABBV':                   Open        High         Low       Close   Adj Close  \
 Date                                                                     
 2021-01-05  105.410004  107.019997  104.629997  106.500000   98.809883   
 2021-01-06  104.750000  107.190002  104.180000  105.580002   97.956322   
 2021-01-07  106.110001  107.059998  105.570000  106.709999   99.004715   
 2021-01-08  106.839996  107.529999  105.760002  107.269997   99.524284   
 2021-01-11  107.070000  109.559998  107.070000  109.019997  101.147919   
 ...                ...         ...         ...         ...         ...   
 2022-03-25  160.550003  161.979996  160.229996  161.330002  158.487930   
 2022-03-28  160.610001  162.029999  159.529999  161.970001  159.116653   
 2022-03-29  162.990005  163.279999  161.289993  162.179993  159.322952   
 2022-03-30  161.720001  164.350006  161.720001  163.750000  160.865295   
 2022-03-31  162.929993  164.660004  162.100006  162.110001  159.254181   
 
             (h

#####Model Logistic Regression 

In [16]:
from sklearn.linear_model import LogisticRegression

In [18]:
model1=LogisticRegression(multi_class='multinomial', solver='lbfgs',max_iter=500)
model2= LogisticRegression(multi_class='multinomial', solver='newton-cg',max_iter=500)

In [19]:
prob1=dict()
prob2=dict()

for j in assets:
    tmp1=[]
    tmp2=[]
    df=stock[j]
    df1=df.dropna()
    X= df1.iloc[:,5:14]
    # print(X) #it has all the days
    y= df1['Rank']
    X_train= X.iloc[:-60] #only the days of one year until 3/01
    # print(X_train)
    Y_train= y.iloc[:-60]
    # print(Y_train)
    model1.fit(X_train,Y_train)
    model2.fit(X_train,Y_train)
    #now each day we will use the models to predict 
    for i in range(59,0,-1): #from 4/01 and later -->predictions
      X_validation=X.iloc[-(i+1):-i] 
      # print(X_validation)
      Y_validation= y.iloc[-(i+1):-i]
      print(Y_validation.index) 
      probability1= model1.predict_proba(X_validation)
      print(probability1[0])
      tmp1.append(probability1[0])
      probability2= model2.predict_proba(X_validation)
      tmp2.append(probability2[0])
    X_validation=X.tail(1) 
    # print(X_validation)
    Y_validation= y.tail(1)
    print(Y_validation.index) 
    probability1= model1.predict_proba(X_validation)
    print(probability1[0])
    tmp1.append(probability1[0])
    probability2= model2.predict_proba(X_validation)
    tmp2.append(probability2[0])
    prob1[j]=tmp1
    prob2[j]=tmp2

Η έξοδος ροής περικόπηκε στις τελευταίες 5000 γραμμές.
DatetimeIndex(['2022-02-02'], dtype='datetime64[ns]', name='Date', freq=None)
[0.10915104 0.26989213 0.18325877 0.2899456  0.14775246]
DatetimeIndex(['2022-02-03'], dtype='datetime64[ns]', name='Date', freq=None)
[0.09537738 0.27535399 0.18069006 0.29762441 0.15095417]
DatetimeIndex(['2022-02-04'], dtype='datetime64[ns]', name='Date', freq=None)
[0.1754385  0.24426439 0.19947236 0.26136022 0.11946453]
DatetimeIndex(['2022-02-07'], dtype='datetime64[ns]', name='Date', freq=None)
[0.19107092 0.23236479 0.19895536 0.22326816 0.15434077]
DatetimeIndex(['2022-02-08'], dtype='datetime64[ns]', name='Date', freq=None)
[0.15410962 0.24287867 0.19781744 0.23033136 0.17486291]
DatetimeIndex(['2022-02-09'], dtype='datetime64[ns]', name='Date', freq=None)
[0.20838646 0.21565992 0.216093   0.18232479 0.17753582]
DatetimeIndex(['2022-02-10'], dtype='datetime64[ns]', name='Date', freq=None)
[0.12033099 0.23741644 0.19311901 0.20110829 0.24802527]


In [20]:
prob1['date']=X.iloc[-60:].index
prob1_final=pd.DataFrame.from_dict(prob1)
prob1_final.set_index('date')
prob2['date']=X.iloc[-60:].index
prob2_final=pd.DataFrame.from_dict(prob2)
prob2_final.set_index('date')

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-01-04,"[0.11451990658566992, 0.20500731722344379, 0.2...","[0.2069516783429683, 0.20980891447960434, 0.21...","[0.13326739936685691, 0.23327195287949123, 0.1...","[0.2082258539467668, 0.21025790322880758, 0.21...","[0.2707458629980206, 0.1421033019172192, 0.103...","[0.1061612094599583, 0.35716337679067495, 0.15...","[0.28190847873733943, 0.14950435810852872, 0.1...","[0.16807898064201038, 0.20848549006875214, 0.1...","[0.23679800951667265, 0.24916597369921592, 0.2...","[0.15654618568132053, 0.2493097329997079, 0.20...",...,"[0.1850811349226256, 0.2716029914657057, 0.236...","[0.29101585138211655, 0.1668952086761903, 0.12...","[0.17391945347716578, 0.1915708719559244, 0.23...","[0.23211050931165625, 0.1907168161648539, 0.18...","[0.2788691532621922, 0.19901376812863297, 0.12...","[0.1657301762052946, 0.19955504439357669, 0.23...","[0.23922237653063597, 0.1836515865248526, 0.15...","[0.20442134704246084, 0.21886052365990305, 0.3...","[0.1586697218912065, 0.12906755319876764, 0.30...","[0.25056389438702537, 0.19971194317008137, 0.1..."
2022-01-05,"[0.15599800336594474, 0.1765179831906511, 0.19...","[0.1716844749005425, 0.22079324198197048, 0.30...","[0.13595064214589778, 0.2271638477273161, 0.17...","[0.15498573410926947, 0.21935378682613785, 0.2...","[0.25831713659688044, 0.1436643064790623, 0.10...","[0.12298174716910514, 0.3220464343685985, 0.16...","[0.2845035622756901, 0.1521244612209724, 0.162...","[0.16672959851949037, 0.18195560943468198, 0.1...","[0.21870414202760036, 0.21379378844121652, 0.2...","[0.19592550824802102, 0.21410959196399462, 0.1...",...,"[0.17646208481011608, 0.2152075245989378, 0.18...","[0.30722969344228596, 0.14375271524975003, 0.1...","[0.15761237269728695, 0.21557358285266418, 0.2...","[0.22863696173773773, 0.18425174446210033, 0.1...","[0.26814720504863604, 0.17917179913901451, 0.0...","[0.14523013698189416, 0.18221987611322485, 0.2...","[0.35633740407537445, 0.15249143040173357, 0.1...","[0.19282653393575877, 0.2008788968654119, 0.30...","[0.17446906645231933, 0.09044938653872113, 0.3...","[0.24432511672565807, 0.20960825985213122, 0.1..."
2022-01-06,"[0.1091521597514361, 0.2035454485473406, 0.264...","[0.21759925757902665, 0.21249613544099505, 0.1...","[0.10812515860831326, 0.28803867053158166, 0.1...","[0.11540737736424019, 0.2474259266007927, 0.19...","[0.3685106712347368, 0.11637216915795588, 0.08...","[0.117496775098581, 0.31178310390651537, 0.173...","[0.2729641532419624, 0.1785448800447536, 0.145...","[0.15582113762559285, 0.18960240405138248, 0.1...","[0.1642167614114406, 0.1961814072947325, 0.268...","[0.17989168293340613, 0.21303548387010804, 0.2...",...,"[0.1883803222630452, 0.2409991882452777, 0.221...","[0.31759400281171873, 0.11740401683201851, 0.0...","[0.1304526283897509, 0.27011260678559573, 0.21...","[0.2399368479163989, 0.20260400169660758, 0.19...","[0.2544334009525382, 0.1747265138314378, 0.122...","[0.17240025534782716, 0.17317552432259964, 0.2...","[0.29451952500817774, 0.16449340862270814, 0.1...","[0.19094271489965367, 0.18336052290823127, 0.2...","[0.15855800848678772, 0.08521640528281495, 0.3...","[0.245119318806454, 0.20841583068653963, 0.106..."
2022-01-07,"[0.12798830257241442, 0.18400874137264608, 0.2...","[0.17741540884459414, 0.22519088565727052, 0.2...","[0.15792984954777525, 0.20741291642737036, 0.1...","[0.14175563865903926, 0.2400020399146275, 0.19...","[0.21395984123212478, 0.15179892299287576, 0.0...","[0.12218784809289897, 0.3083996265539207, 0.17...","[0.2756189161032056, 0.1663488702908075, 0.172...","[0.14814389461842034, 0.15991695906550016, 0.1...","[0.1791236153994423, 0.20433892476214877, 0.25...","[0.16774597077176287, 0.22053513649072107, 0.2...",...,"[0.17878543772418842, 0.2517942993004373, 0.23...","[0.2930875883661652, 0.15010731086030893, 0.12...","[0.12008017042176641, 0.27279589258125714, 0.2...","[0.23298430677199922, 0.1937

######SOLVER 1
***

In [21]:
RPS=pd.DataFrame()

# RPS=RPS.drop('date',axis=1)
# print(RPS.index[2])
for k in prob1_final.index: #for everyday of a stock->index
  prob=pd.DataFrame()
  prob1=pd.DataFrame()
  predic=pd.DataFrame()
  if(prob1_final['date'][k]!=prob1_final['date'].tail(1).values): #if it's not for the last day 
    #for historical data of two days 
    prob=rd.loc[(rd.index.date==prob1_final['date'][k])]
    prob1=rd.loc[ (rd.index.date==prob1_final['date'][k+1])]
    prob=create_historical(prob)
    prob1=create_historical(prob1)
    hist_data=prob.append(prob1)
    # print(hist_data)
    predic=prob1_final.loc[prob1_final['date']==prob1_final['date'][k]] #find predictions of this day
    predic=predic.drop('date',axis=1)
    # print(predic)
    data=pd.DataFrame(index=predic.columns)
    data=data.reset_index()
    # print(data)
    data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
    for i in range(len(np.array_split(predic.values[0],46))-1):
      # print(i)
      # print(np.array_split(predic.values[0],46)[i][0])
      data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
    data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
    # print(data)
    #now we have data->predictions and hist_data->historical data so now we can call the RPS
    rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
    # print(rps) ->rps for every stock for one day 
    # print(rps['RPS'])
    tmp=pd.DataFrame(rps['RPS'].values).T
    # print(tmp.T)
    RPS=RPS.append(tmp,ignore_index=True)
    # print(RPS[RPS.index==k])

    
  else:
    #for the last day
    prob=rd.loc[(rd.index.date==prob1_final['date'][k])]
    prob1=rd.loc[ (rd.index.date==rd.index.date[-1])]
    prob=create_historical(prob)
    prob1=create_historical(prob1)
    hist_data=prob.append(prob1)
    # print(hist_data)
    predic=prob1_final.loc[prob1_final['date']==prob1_final['date'][k]] #find predictions of this day
    predic=predic.drop('date',axis=1)
    # print(predic)
    data=pd.DataFrame(index=predic.columns)
    data=data.reset_index()
    # print(data)
    data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
    for i in range(len(np.array_split(predic.values[0],46))-1):
      # print(i)
      # print(np.array_split(predic.values[0],46)[i][0])
      data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
    data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
    # print(data)
    #now we have data->predictions and hist_data->historical data so now we can call the RPS
    rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
    # print(rps) ->rps for every stock for one day 
    # print(rps['RPS'])
    tmp=pd.DataFrame(rps['RPS'].values).T
    # print(tmp.T)
    RPS=RPS.append(tmp,ignore_index=True)              
print(RPS)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == dat

          0         1         2         3         4         5         6   \
0   0.207296  0.276089  0.090933  0.109802  0.249597  0.126248  0.104309   
1   0.149639  0.099568  0.130542  0.267039  0.173454  0.123140  0.121897   
2   0.067546  0.264453  0.259943  0.131117  0.188133  0.117335  0.097844   
3   0.201159  0.059611  0.262438  0.122122  0.180455  0.076393  0.117184   
4   0.150883  0.219676  0.075249  0.096903  0.113346  0.252290  0.250839   
5   0.068461  0.124331  0.110045  0.109250  0.202468  0.122852  0.117278   
6   0.279489  0.285580  0.107900  0.123293  0.129372  0.129001  0.214223   
7   0.228302  0.133410  0.066868  0.125246  0.183029  0.119933  0.126335   
8   0.174591  0.127918  0.063155  0.101605  0.156123  0.255860  0.118494   
9   0.144112  0.129701  0.056719  0.077633  0.267478  0.263697  0.212126   
10  0.127666  0.125989  0.092154  0.267319  0.330134  0.070072  0.212307   
11  0.101730  0.273558  0.050508  0.231387  0.218464  0.288472  0.211924   
12  0.159506

In [22]:
d=prob1_final['date']
name=prob1_final.columns[:-1]
# print(np.split(name.values,len(name)))
# print(d.values)
RPS['date']=d.values
RPS=RPS.set_index('date')
RPS.columns=assets
RPS

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-01-04,0.207296,0.276089,0.090933,0.109802,0.249597,0.126248,0.104309,0.081447,0.160769,0.115418,...,0.090361,0.123569,0.073934,0.223945,0.197506,0.108577,0.240059,0.063077,0.278159,0.226773
2022-01-05,0.149639,0.099568,0.130542,0.267039,0.173454,0.123140,0.121897,0.079113,0.080820,0.119301,...,0.113761,0.228052,0.227237,0.240010,0.128385,0.278988,0.114542,0.103746,0.277569,0.222072
2022-01-06,0.067546,0.264453,0.259943,0.131117,0.188133,0.117335,0.097844,0.146331,0.065462,0.116265,...,0.259039,0.222202,0.229808,0.138701,0.227380,0.126450,0.208205,0.112287,0.294324,0.131429
2022-01-07,0.201159,0.059611,0.262438,0.122122,0.180455,0.076393,0.117184,0.084937,0.244066,0.073753,...,0.072726,0.214028,0.120559,0.129722,0.267931,0.251067,0.125541,0.236182,0.082978,0.105761
2022-01-10,0.150883,0.219676,0.075249,0.096903,0.113346,0.252290,0.250839,0.142085,0.228564,0.079671,...,0.079713,0.231773,0.115926,0.222626,0.166352,0.064827,0.196399,0.119059,0.067259,0.236974
2022-01-11,0.068461,0.124331,0.110045,0.109250,0.202468,0.122852,0.117278,0.077473,0.186428,0.117339,...,0.083119,0.220008,0.271653,0.244846,0.219021,0.239569,0.273585,0.099858,0.295283,0.130258
2022-01-12,0.279489,0.285580,0.107900,0.123293,0.129372,0.129001,0.214223,0.102470,0.078681,0.086559,...,0.224888,0.213341,0.229240,0.112416,0.108081,0.141679,0.227605,0.061826,0.289413,0.101955
2022-01-13,0.228302,0.133410,0.066868,0.125246,0.183029,0.119933,0.126335,0.088571,0.106767,0.230976,...,0.262580,0.132010,0.229677,0.224640,0.239490,0.107992,0.238705,0.062482,0.289669,0.230824
2022-01-14,0.174591,0.127918,0.063155,0.101605,0.156123,0.255860,0.118494,0.081249,0.117973,0.088719,...,0.071708,0.139106,0.232883,0.114431,0.279043,0.065028,0.194426,0.241553,0.238973,0.205080


######SOLVER 2
***

In [23]:
#--- do the same but with solver 2 -->prob2_final---#
RPS2=pd.DataFrame()

# RPS2=RPS2.drop('date',axis=1)
# print(RPS2.index[2])
for k in prob2_final.index: #for everyday of a stock->index
  prob=pd.DataFrame()
  prob1=pd.DataFrame()
  predic=pd.DataFrame()
  if(prob2_final['date'][k]!=prob2_final['date'].tail(1).values): #if it's not for the last day 
    #for historical data of two days 
    prob=rd.loc[(rd.index.date==prob2_final['date'][k])]
    prob1=rd.loc[ (rd.index.date==prob2_final['date'][k+1])]
    prob=create_historical(prob)
    prob1=create_historical(prob1)
    hist_data=prob.append(prob1)
    # print(hist_data)
    predic=prob2_final.loc[prob2_final['date']==prob2_final['date'][k]] #find predictions of this day
    predic=predic.drop('date',axis=1)
    # print(predic)
    data=pd.DataFrame(index=predic.columns)
    data=data.reset_index()
    # print(data)
    data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
    for i in range(len(np.array_split(predic.values[0],46))-1):
      # print(i)
      # print(np.array_split(predic.values[0],46)[i][0])
      data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
    data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
    # print(data)
    #now we have data->predictions and hist_data->historical data so now we can call the RPS2
    rps2=RPS_calculation(hist_data = hist_data , submission = data)['details']
    # print(RPS2) ->RPS2 for every stock for one day 
    # print(RPS2['RPS'])
    tmp=pd.DataFrame(rps2['RPS'].values).T
    # print(tmp.T)
    RPS2=RPS2.append(tmp,ignore_index=True)
    # print(RPS2[RPS2.index==k])

    
  else:
    #for the last day
    prob=rd.loc[(rd.index.date==prob2_final['date'][k])]
    prob1=rd.loc[ (rd.index.date==rd.index.date[-1])]
    prob=create_historical(prob)
    prob1=create_historical(prob1)
    hist_data=prob.append(prob1)
    # print(hist_data)
    predic=prob2_final.loc[prob2_final['date']==prob2_final['date'][k]] #find predictions of this day
    predic=predic.drop('date',axis=1)
    # print(predic)
    data=pd.DataFrame(index=predic.columns)
    data=data.reset_index()
    # print(data)
    data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
    for i in range(len(np.array_split(predic.values[0],46))-1):
      # print(i)
      # print(np.array_split(predic.values[0],46)[i][0])
      data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
    data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
    # print(data)
    #now we have data->predictions and hist_data->historical data so now we can call the RPS2
    rps2=RPS_calculation(hist_data = hist_data , submission = data)['details']
    # print(RPS2) ->RPS2 for every stock for one day 
    # print(RPS2['RPS2'])
    tmp=pd.DataFrame(rps2['RPS'].values).T
    # print(tmp.T)
    RPS2=RPS2.append(tmp,ignore_index=True)              
print(RPS2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


          0         1         2         3         4         5         6   \
0   0.207156  0.276100  0.090985  0.109773  0.249631  0.126381  0.104084   
1   0.149597  0.099599  0.130546  0.267121  0.173516  0.123177  0.122030   
2   0.067543  0.264519  0.260148  0.131210  0.188042  0.117371  0.097654   
3   0.201089  0.059633  0.262584  0.122147  0.180266  0.076359  0.117266   
4   0.150824  0.219714  0.075164  0.096797  0.113466  0.252261  0.250697   
5   0.068455  0.124333  0.109961  0.109179  0.202696  0.122877  0.117351   
6   0.279580  0.285545  0.107727  0.123341  0.129393  0.129014  0.214549   
7   0.228127  0.133385  0.066827  0.125306  0.183066  0.119904  0.126061   
8   0.174849  0.127918  0.063117  0.101525  0.156264  0.255893  0.118582   
9   0.144048  0.129695  0.056690  0.077540  0.267472  0.263899  0.212388   
10  0.127747  0.126013  0.092155  0.267465  0.329511  0.070143  0.212396   
11  0.101683  0.273599  0.050547  0.231468  0.217922  0.288549  0.211836   
12  0.159384

In [24]:
d2=prob2_final['date']
name2=prob2_final.columns[:-1]
RPS2['date']=d2.values
RPS2=RPS2.set_index('date')
RPS2.columns=assets
RPS2

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-01-04,0.207156,0.276100,0.090985,0.109773,0.249631,0.126381,0.104084,0.081438,0.160604,0.115469,...,0.090504,0.123379,0.073917,0.223956,0.197610,0.108570,0.240001,0.063043,0.278093,0.226025
2022-01-05,0.149597,0.099599,0.130546,0.267121,0.173516,0.123177,0.122030,0.079094,0.080983,0.119390,...,0.113624,0.228394,0.227276,0.239932,0.128430,0.278879,0.114455,0.103771,0.277010,0.221704
2022-01-06,0.067543,0.264519,0.260148,0.131210,0.188042,0.117371,0.097654,0.146226,0.065607,0.116299,...,0.258977,0.221897,0.230055,0.138642,0.227450,0.126391,0.208189,0.112281,0.293850,0.131317
2022-01-07,0.201089,0.059633,0.262584,0.122147,0.180266,0.076359,0.117266,0.084854,0.244101,0.073773,...,0.072740,0.213381,0.120751,0.129681,0.268010,0.251179,0.125491,0.236129,0.083200,0.105722
2022-01-10,0.150824,0.219714,0.075164,0.096797,0.113466,0.252261,0.250697,0.142025,0.228422,0.079683,...,0.079680,0.232126,0.116099,0.222687,0.166370,0.064972,0.196442,0.119123,0.067242,0.236660
2022-01-11,0.068455,0.124333,0.109961,0.109179,0.202696,0.122877,0.117351,0.077643,0.185932,0.117283,...,0.083082,0.219705,0.271337,0.244733,0.218782,0.239596,0.273621,0.099907,0.295157,0.130112
2022-01-12,0.279580,0.285545,0.107727,0.123341,0.129393,0.129014,0.214549,0.102476,0.078658,0.086576,...,0.225238,0.212753,0.229115,0.112489,0.108090,0.141666,0.227719,0.061817,0.288734,0.101971
2022-01-13,0.228127,0.133385,0.066827,0.125306,0.183066,0.119904,0.126061,0.088557,0.106833,0.230840,...,0.263041,0.131896,0.229969,0.224629,0.239311,0.107953,0.238638,0.062492,0.288913,0.231042
2022-01-14,0.174849,0.127918,0.063117,0.101525,0.156264,0.255893,0.118582,0.081386,0.117893,0.088689,...,0.071784,0.139064,0.233236,0.114481,0.279031,0.065094,0.194420,0.241611,0.239437,0.205624


#####Final RPS calculation for Logistic Regression

In [26]:
print(RPS.mean(axis=1).mean())
print(RPS2.mean(axis=1).mean())

0.15973738477770036
0.15973583679836742


#####Model Random Forest

Random forest algorithm can be used for both classifications and regression task. It provides higher accuracy through cross validation. Random forest classifier will handle the missing values and maintain the accuracy of a large proportion of data.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model3=RandomForestClassifier(n_estimators=100)

In [ ]:
prob3=dict()

for j in assets:
    tmp3=[]
    df=stock[j]
    df1=df.dropna()
    X= df1.iloc[:,5:14]
    # print(X) #it has all the days
    y= df1['Rank']
    X_train= X.iloc[:-60] #only the days of one year until 3/01
    # print(X_train)
    Y_train= y.iloc[:-60]
    # print(Y_train)
    model3.fit(X_train,Y_train)
    #now each day we will use the models to predict 
    for i in range(59,0,-1): #from 4/01 and later -->predictions
      X_validation=X.iloc[-(i+1):-i] 
      # print(X_validation)
      Y_validation= y.iloc[-(i+1):-i]
      print(Y_validation.index) 
      probability3= model3.predict_proba(X_validation)
      print(probability3[0])
      tmp3.append(probability3[0])
    X_validation=X.tail(1) 
    # print(X_validation)
    Y_validation= y.tail(1)
    print(Y_validation.index) 
    probability3= model3.predict_proba(X_validation)
    print(probability3[0])
    tmp3.append(probability3[0])
    prob3[j]=tmp3
   

Streaming output truncated to the last 5000 lines.
DatetimeIndex(['2022-02-02'], dtype='datetime64[ns]', name='Date', freq=None)
[0.05 0.32 0.07 0.47 0.09]
DatetimeIndex(['2022-02-03'], dtype='datetime64[ns]', name='Date', freq=None)
[0.11 0.25 0.17 0.31 0.16]
DatetimeIndex(['2022-02-04'], dtype='datetime64[ns]', name='Date', freq=None)
[0.08 0.32 0.12 0.17 0.31]
DatetimeIndex(['2022-02-07'], dtype='datetime64[ns]', name='Date', freq=None)
[0.04 0.36 0.07 0.38 0.15]
DatetimeIndex(['2022-02-08'], dtype='datetime64[ns]', name='Date', freq=None)
[0.28 0.13 0.18 0.32 0.09]
DatetimeIndex(['2022-02-09'], dtype='datetime64[ns]', name='Date', freq=None)
[0.13 0.27 0.12 0.29 0.19]
DatetimeIndex(['2022-02-10'], dtype='datetime64[ns]', name='Date', freq=None)
[0.13 0.22 0.11 0.19 0.35]
DatetimeIndex(['2022-02-11'], dtype='datetime64[ns]', name='Date', freq=None)
[0.15 0.48 0.13 0.11 0.13]
DatetimeIndex(['2022-02-14'], dtype='datetime64[ns]', name='Date', freq=None)
[0.12 0.31 0.16 0.32 0.09]
Date

In [ ]:
prob3['date']=X.iloc[-60:].index
prob3_final=pd.DataFrame.from_dict(prob3)
prob3_final.set_index('date')


,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-01-04,"[0.12, 0.33, 0.19, 0.13, 0.23]","[0.13, 0.21, 0.26, 0.3, 0.1]","[0.08, 0.1, 0.15, 0.39, 0.28]","[0.32, 0.23, 0.25, 0.06, 0.14]","[0.43, 0.1, 0.03, 0.03, 0.41]","[0.06, 0.13, 0.35, 0.09, 0.37]","[0.35, 0.04, 0.16, 0.3, 0.15]","[0.12, 0.24, 0.23, 0.22, 0.19]","[0.2, 0.31, 0.17, 0.08, 0.24]","[0.14, 0.15, 0.41, 0.2, 0.1]",...,"[0.24, 0.2, 0.07, 0.23, 0.26]","[0.35, 0.2, 0.07, 0.2, 0.18]","[0.22, 0.22, 0.35, 0.04, 0.17]","[0.25, 0.08, 0.17, 0.35, 0.15]","[0.19, 0.27, 0.12, 0.25, 0.17]","[0.2, 0.29, 0.26, 0.09, 0.16]","[0.13, 0.18, 0.08, 0.23, 0.38]","[0.32, 0.23, 0.37, 0.04, 0.04]","[0.16, 0.18, 0.27, 0.28, 0.11]","[0.21, 0.1, 0.22, 0.35, 0.12]"
2022-01-05,"[0.05, 0.13, 0.21, 0.27, 0.34]","[0.19, 0.31, 0.18, 0.24, 0.08]","[0.11, 0.12, 0.11, 0.34, 0.32]","[0.35, 0.13, 0.13, 0.18, 0.21]","[0.36, 0.07, 0.03, 0.06, 0.48]","[0.15, 0.4, 0.05, 0.27, 0.13]","[0.35, 0.03, 0.18, 0.11, 0.33]","[0.17, 0.19, 0.19, 0.32, 0.13]","[0.17, 0.24, 0.26, 0.17, 0.16]","[0.13, 0.24, 0.11, 0.48, 0.04]",...,"[0.09, 0.17, 0.33, 0.12, 0.29]","[0.23, 0.08, 0.12, 0.12, 0.45]","[0.1, 0.33, 0.38, 0.06, 0.13]","[0.25, 0.25, 0.24, 0.18, 0.08]","[0.15, 0.4, 0.1, 0.16, 0.19]","[0.19, 0.4, 0.29, 0.04, 0.08]","[0.13, 0.18, 0.05, 0.19, 0.45]","[0.13, 0.23, 0.52, 0.1, 0.02]","[0.16, 0.09, 0.44, 0.23, 0.08]","[0.21, 0.24, 0.06, 0.22, 0.27]"
2022-01-06,"[0.05, 0.16, 0.29, 0.26, 0.24]","[0.31, 0.23, 0.16, 0.28, 0.02]","[0.11, 0.24, 0.28, 0.19, 0.18]","[0.18, 0.38, 0.15, 0.23, 0.06]","[0.32, 0.2, 0.06, 0.11, 0.31]","[0.06, 0.31, 0.12, 0.27, 0.24]","[0.37, 0.16, 0.3, 0.08, 0.09]","[0.16, 0.26, 0.15, 0.29, 0.14]","[0.05, 0.09, 0.24, 0.48, 0.14]","[0.15, 0.23, 0.37, 0.16, 0.09]",...,"[0.14, 0.12, 0.17, 0.1, 0.47]","[0.47, 0.14, 0.01, 0.16, 0.22]","[0.13, 0.64, 0.06, 0.07, 0.1]","[0.24, 0.22, 0.31, 0.19, 0.04]","[0.26, 0.21, 0.05, 0.08, 0.4]","[0.23, 0.41, 0.13, 0.14, 0.09]","[0.35, 0.16, 0.12, 0.05, 0.32]","[0.09, 0.08, 0.34, 0.48, 0.01]","[0.08, 0.16, 0.23, 0.44, 0.09]","[0.11, 0.42, 0.1, 0.05, 0.32]"
2022-01-07,"[0.1, 0.26, 0.25, 0.17, 0.22]","[0.29, 0.27, 0.21, 0.18, 0.05]","[0.17, 0.12, 0.19, 0.38, 0.14]","[0.09, 0.2, 0.33, 0.28, 0.1]","[0.15, 0.25, 0.02, 0.25, 0.33]","[0.03, 0.2, 0.09, 0.58, 0.1]","[0.48, 0.06, 0.18, 0.12, 0.16]","[0.06, 0.15, 0.26, 0.38, 0.15]","[0.13, 0.32, 0.17, 0.17, 0.21]","[0.16, 0.17, 0.25, 0.15, 0.27]",...,"[0.09, 0.23, 0.15, 0.23, 0.3]","[0.28, 0.05, 0.02, 0.13, 0.52]","[0.04, 0.36, 0.2, 0.11, 0.29]","[0.12, 0.19, 0.32, 0.23, 0.14]","[0.32, 0.19, 0.08, 0.09, 0.32]","[0.18, 0.12, 0.21, 0.19, 0.3]","[0.24, 0.21, 0.13, 0.11, 0.31]","[0.14, 0.14, 0.45, 0.19, 0.08]","[0.13, 0.19, 0.3, 0.22, 0.16]","[0.37, 0.09, 0.13, 0.33, 0.08]"
2022-01-10,"[0.13, 0.2, 0.33, 0.12, 0.22]","[0.34, 0.26, 0.09, 0.16, 0.15]","[0.05, 0.26, 0.15, 0.32, 0.22]","[0.05, 0.42, 0.23, 0.23, 0.07]","[0.33, 0.11, 0.05, 0.16, 0.35]","[0.15, 0.32, 0.09, 0.29, 0.15]","[0.46, 0.06, 0.18, 0.06, 0.24]","[0.08, 0.19, 0.29, 0.24, 0.2]","[0.02, 0.07, 0.47, 0.29, 0.15]","[0.18, 0.13, 0.16, 0.32, 0.21]",...,"[0.4, 0.18, 0.13, 0.09, 0.2]","[0.29, 0.18, 0.05, 0.3, 0.18]","[0.05, 0.4, 0.1, 0.1, 0.35]","[0.13, 0.26, 0.21, 0.32, 0.08]","[0.03, 0.09, 0.12, 0.3, 0.46]","[0.13, 0.09, 0.16, 0.23, 0.39]","[0.21, 0.18, 0.07, 0.11, 0.43]","[0.25, 0.16, 0.21, 0.31, 0.07]","[0.05, 0.32, 0.23, 0.21, 0.19]","[0.55, 0.2, 0.07, 0.1, 0.08]"
2022-01-11,"[0.05, 0.29, 0.23, 0.24, 0.19]","[0.4, 0.23, 0.05, 0.23, 0.09]","[0.17, 0.18, 0.3, 0.16, 0.19]","[0.18, 0.53, 0.12, 0.1, 0.07]","[0.18, 0.16, 0.03, 0.15, 0.48]","[0.12, 0.28, 0.13, 0.28, 0.19]","[0.24, 0.21, 0.14, 0.11, 0.3]","[0.09, 0.3, 0.11, 0.25, 0.25]","[0.46, 0.17, 0.12, 0.11, 0.14]","[0.13, 0.13, 0.16, 0.3, 0.28]",...,"[0.22, 0.25, 0.16, 0.1, 0.27]","[0.54, 0.11, 0.03, 0.15, 0.17]","[0.13, 0.43, 0.06, 0.13, 0.25]","[0.24, 0.13, 0.18, 0.22, 0.23]","[0.09, 0.12, 0.35, 0.2, 0.24]","[0.08, 0.26, 0.2, 0.12, 0.34]","[0.11, 0.15, 0.15, 0

In [ ]:
RPS3=pd.DataFrame()

for k in prob3_final.index: #for everyday of a stock->index
  prob=pd.DataFrame()
  prob1=pd.DataFrame()
  predic=pd.DataFrame()
  if(prob3_final['date'][k]!=prob3_final['date'].tail(1).values): #if it's not for the last day 
    #for historical data of two days 
    prob=rd.loc[(rd.index.date==prob1_final['date'][k])]
    prob1=rd.loc[ (rd.index.date==prob1_final['date'][k+1])]
    prob=create_historical(prob)
    prob1=create_historical(prob1)
    hist_data=prob.append(prob1)
    # print(hist_data)
    predic=prob3_final.loc[prob3_final['date']==prob3_final['date'][k]] #find predictions of this day
    predic=predic.drop('date',axis=1)
    # print(predic)
    data=pd.DataFrame(index=predic.columns)
    data=data.reset_index()
    # print(data)
    data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
    for i in range(len(np.array_split(predic.values[0],46))-1):
      # print(i)
      # print(np.array_split(predic.values[0],46)[i][0])
      data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
    data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
    # print(data)
    #now we have data->predictions and hist_data->historical data so now we can call the RPS
    rps3=RPS_calculation(hist_data = hist_data , submission = data)['details']
    # print(rps) ->rps for every stock for one day 
    # print(rps['RPS'])
    tmp=pd.DataFrame(rps3['RPS'].values).T
    # print(tmp.T)
    RPS3=RPS3.append(tmp,ignore_index=True)
    # print(RPS[RPS.index==k])

    
  else:
    #for the last day
    prob=rd.loc[(rd.index.date==prob1_final['date'][k])]
    prob1=rd.loc[ (rd.index.date==rd.index.date[-1])]
    prob=create_historical(prob)
    prob1=create_historical(prob1)
    hist_data=prob.append(prob1)
    # print(hist_data)
    predic=prob3_final.loc[prob3_final['date']==prob3_final['date'][k]] #find predictions of this day
    predic=predic.drop('date',axis=1)
    # print(predic)
    data=pd.DataFrame(index=predic.columns)
    data=data.reset_index()
    # print(data)
    data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
    for i in range(len(np.array_split(predic.values[0],46))-1):
      # print(i)
      # print(np.array_split(predic.values[0],46)[i][0])
      data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
    data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
    # print(data)
    #now we have data->predictions and hist_data->historical data so now we can call the RPS
    rps3=RPS_calculation(hist_data = hist_data , submission = data)['details']
    # print(rps) ->rps for every stock for one day 
    # print(rps['RPS'])
    tmp=pd.DataFrame(rps3['RPS'].values).T
    # print(tmp.T)
    RPS3=RPS3.append(tmp,ignore_index=True)              
print(RPS3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == dat

         0        1        2        3        4        5        6        7   \
0   0.24388  0.26050  0.04522  0.07290  0.18150  0.09364  0.09992  0.06964   
1   0.23252  0.07898  0.22860  0.17782  0.15930  0.14038  0.16188  0.07558   
2   0.07084  0.36762  0.24078  0.06274  0.19306  0.08764  0.09096  0.11330   
3   0.22402  0.09062  0.29660  0.13332  0.16556  0.10524  0.10920  0.07022   
4   0.12596  0.14284  0.08772  0.14366  0.13302  0.25590  0.30992  0.15458   
5   0.06782  0.20548  0.12200  0.03006  0.11106  0.09828  0.12364  0.09454   
6   0.24782  0.25140  0.14390  0.12268  0.18068  0.20700  0.15034  0.14740   
7   0.17812  0.11652  0.07260  0.08406  0.18444  0.10300  0.12626  0.08316   
8   0.24366  0.14468  0.09302  0.11420  0.26450  0.26572  0.14250  0.08300   
9   0.12090  0.15266  0.09582  0.09170  0.15642  0.23286  0.24814  0.26238   
10  0.12620  0.03366  0.06918  0.34796  0.40650  0.03800  0.26484  0.13284   
11  0.11630  0.24756  0.07284  0.20522  0.21140  0.32102  0.2373

In [ ]:
d3=prob3_final['date']
name3=prob3_final.columns[:-1]
# print(np.split(name.values,len(name)))
# print(d.values)
RPS3['date']=d3.values
RPS3=RPS3.set_index('date')
RPS3.columns=assets
RPS3

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-01-04,0.24388,0.26050,0.04522,0.07290,0.18150,0.09364,0.09992,0.06964,0.16402,0.12074,...,0.13578,0.10036,0.06300,0.25678,0.23060,0.17364,0.34990,0.08258,0.26108,0.23910
2022-01-05,0.23252,0.07898,0.22860,0.17782,0.15930,0.14038,0.16188,0.07558,0.06630,0.07716,...,0.10158,0.12728,0.32158,0.34130,0.07672,0.16900,0.14502,0.08826,0.27412,0.20792
2022-01-06,0.07084,0.36762,0.24078,0.06274,0.19306,0.08764,0.09096,0.11330,0.08522,0.14750,...,0.11060,0.12516,0.42174,0.17274,0.24378,0.04870,0.18038,0.18744,0.34260,0.15846
2022-01-07,0.22402,0.09062,0.29660,0.13332,0.16556,0.10524,0.10920,0.07022,0.24958,0.07676,...,0.09628,0.33204,0.12114,0.10540,0.19460,0.17450,0.12652,0.26746,0.10586,0.10460
2022-01-10,0.12596,0.14284,0.08772,0.14366,0.13302,0.25590,0.30992,0.15458,0.20892,0.09070,...,0.12410,0.24956,0.12600,0.25908,0.07290,0.12036,0.14654,0.12398,0.06710,0.47676
2022-01-11,0.06782,0.20548,0.12200,0.03006,0.11106,0.09828,0.12364,0.09454,0.23812,0.19586,...,0.09582,0.09308,0.23148,0.21798,0.34068,0.16984,0.36798,0.10180,0.28550,0.08060
2022-01-12,0.24782,0.25140,0.14390,0.12268,0.18068,0.20700,0.15034,0.14740,0.09670,0.09990,...,0.21042,0.32028,0.21700,0.19958,0.12308,0.23624,0.22236,0.09624,0.24776,0.15026
2022-01-13,0.17812,0.11652,0.07260,0.08406,0.18444,0.10300,0.12626,0.08316,0.09412,0.33570,...,0.35820,0.12570,0.23284,0.33318,0.14056,0.08556,0.40242,0.06894,0.25730,0.43378
2022-01-14,0.24366,0.14468,0.09302,0.11420,0.26450,0.26572,0.14250,0.08300,0.15830,0.10364,...,0.09068,0.09434,0.18750,0.12120,0.34844,0.08220,0.29208,0.32828,0.28502,0.26804


#####Final RPS calculation for Random Forest

In [ ]:
print(RPS3.mean(axis=1).mean())

0.1677988074074074


#####Model Gradient Boosting


Gradient boosting classifiers are a group of machine learning algorithms that combine many weak learning models together to create a strong predictive model. Decision trees are usually used when doing gradient boosting. Gradient boosting models are becoming popular because of their effectiveness at classifying complex datasets, and have recently been used to win many Kaggle data science competitions.

The Python machine learning library, Scikit-Learn, supports different implementations of gradient boosting classifiers, including XGBoost.

The objective of Gradient Boosting classifiers is to minimize the loss, or the difference between the actual class value of the training example and the predicted class value. It isn't required to understand the process for reducing the classifier's loss, but it operates similarly to gradient descent in a neural network.

**Tuning the model's hyperparameters requires some active decision making on our part. There are various arguments/hyperparameters we can tune to try and get the best accuracy for the model. One of the ways we can do this is by altering the learning rate of the model. We'll want to check the performance of the model on the training set at different learning rates, and then use the best learning rate to make predictions.**

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
model4=GradientBoostingClassifier(n_estimators=60)

In [ ]:
prob4=dict()

for j in assets:
    tmp4=[]
    df=stock[j]
    df1=df.dropna()
    X= df1.iloc[:,5:14]
    # print(X) #it has all the days
    y= df1['Rank']
    X_train= X.iloc[:-60] #only the days of one year until 3/01
    # print(X_train)
    Y_train= y.iloc[:-60]
    # print(Y_train)
    model4.fit(X_train,Y_train)
    #now each day we will use the models to predict 
    for i in range(59,0,-1): #from 4/01 and later -->predictions
      X_validation=X.iloc[-(i+1):-i] 
      # print(X_validation)
      Y_validation= y.iloc[-(i+1):-i]
      print(Y_validation.index) 
      probability4= model4.predict_proba(X_validation)
      print(probability4[0])
      tmp4.append(probability4[0])
    X_validation=X.tail(1) 
    # print(X_validation)
    Y_validation= y.tail(1)
    print(Y_validation.index) 
    probability4= model4.predict_proba(X_validation)
    print(probability4[0])
    tmp4.append(probability4[0])
    prob4[j]=tmp4

Streaming output truncated to the last 5000 lines.
[0.07811399 0.22281307 0.08652723 0.50426833 0.10827738]
DatetimeIndex(['2022-02-03'], dtype='datetime64[ns]', name='Date', freq=None)
[0.07679531 0.2108573  0.12367159 0.36857184 0.22010395]
DatetimeIndex(['2022-02-04'], dtype='datetime64[ns]', name='Date', freq=None)
[0.26077017 0.25094563 0.02873413 0.06236176 0.39718832]
DatetimeIndex(['2022-02-07'], dtype='datetime64[ns]', name='Date', freq=None)
[0.06688051 0.24127724 0.09237612 0.51628377 0.08318236]
DatetimeIndex(['2022-02-08'], dtype='datetime64[ns]', name='Date', freq=None)
[0.24689198 0.21370528 0.20466046 0.28803859 0.04670369]
DatetimeIndex(['2022-02-09'], dtype='datetime64[ns]', name='Date', freq=None)
[0.07167979 0.37075407 0.08466492 0.13758027 0.33532095]
DatetimeIndex(['2022-02-10'], dtype='datetime64[ns]', name='Date', freq=None)
[0.13221473 0.23522802 0.07085766 0.06718842 0.49451118]
DatetimeIndex(['2022-02-11'], dtype='datetime64[ns]', name='Date', freq=None)
[0.0

In [ ]:
prob4['date']=X.iloc[-60:].index
prob4_final=pd.DataFrame.from_dict(prob4)
prob4_final.set_index('date')

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-01-04,"[0.07194361624252003, 0.3420255231001213, 0.13...","[0.1355124097295614, 0.1951329019458445, 0.401...","[0.05134771973101992, 0.06104089177650249, 0.1...","[0.6108752730705863, 0.0795004363393534, 0.184...","[0.22871009999921682, 0.07867607321059579, 0.0...","[0.03799349907573387, 0.1387149785879732, 0.23...","[0.4837469237663666, 0.05594483986765017, 0.22...","[0.1490138471462594, 0.19575813507988318, 0.16...","[0.3669520280157783, 0.03649112962211288, 0.05...","[0.058081367037396546, 0.1549468444308143, 0.2...",...,"[0.2775568874092336, 0.06946709882718874, 0.03...","[0.3323907927282086, 0.17803252608530748, 0.02...","[0.06518598230979986, 0.3660227348760848, 0.37...","[0.11787422498138533, 0.05776967469828189, 0.1...","[0.2457054635800479, 0.2665034819043279, 0.119...","[0.15168501932033246, 0.19361389472904747, 0.5...","[0.24639171841687565, 0.16848585617465195, 0.2...","[0.22652731254583647, 0.25878998826630295, 0.3...","[0.24347569193646662, 0.2111302517400386, 0.28...","[0.1763626535808446, 0.04517599376975285, 0.08..."
2022-01-05,"[0.0258344547034365, 0.3785456350280469, 0.109...","[0.2502818622258848, 0.218753358172972, 0.3493...","[0.06653398871944614, 0.036509882199857874, 0....","[0.481571344030679, 0.08722644823280737, 0.087...","[0.2459543155369185, 0.11760940669267461, 0.02...","[0.7181832562329262, 0.21759592920593468, 0.00...","[0.6206439193446623, 0.03261421955860499, 0.07...","[0.1581442836686606, 0.06290651252854097, 0.34...","[0.3254133522541908, 0.13830590007880816, 0.24...","[0.08143353674437986, 0.17235024428946538, 0.0...",...,"[0.0161749202468725, 0.597555017462073, 0.0062...","[0.2249696367133681, 0.19370268460172713, 0.05...","[0.027226538860255195, 0.1078417489678505, 0.6...","[0.14744383769790378, 0.17974143219576375, 0.0...","[0.24726743563349318, 0.3807532366531445, 0.06...","[0.14967972044761768, 0.353191123330056, 0.379...","[0.05920918531100413, 0.04432998645849626, 0.0...","[0.12472455219264321, 0.15032930270740852, 0.5...","[0.05264742050761747, 0.012631758603256316, 0....","[0.3592762836895626, 0.4222024499485044, 0.018..."
2022-01-06,"[0.026002130987119156, 0.14699878248207043, 0....","[0.09234533291608524, 0.5237229580083302, 0.06...","[0.04718208668625685, 0.06962072404958076, 0.3...","[0.16637645582524854, 0.2365429299802369, 0.17...","[0.46360088945745576, 0.15819042490201332, 0.0...","[0.01897104811080187, 0.24932565736118273, 0.0...","[0.1201481998131573, 0.06548154386692447, 0.77...","[0.07880681811748315, 0.14473038395598423, 0.2...","[0.048303188913507315, 0.03223226100177328, 0....","[0.12446219053971157, 0.28506453839346607, 0.3...",...,"[0.09247300734527367, 0.1393579850159622, 0.24...","[0.41810720800937284, 0.1254065961479823, 0.01...","[0.22366339414659467, 0.5735740384417048, 0.04...","[0.11617637781101531, 0.11856805398596028, 0.6...","[0.2681384357975794, 0.23498486835280757, 0.08...","[0.12020910102169173, 0.7480440906907826, 0.07...","[0.5199993272302833, 0.2061569947224092, 0.071...","[0.1658242953506557, 0.17334680192704757, 0.29...","[0.04899314482609248, 0.4243091172033779, 0.25...","[0.12793631190933274, 0.775122615636636, 0.005..."
2022-01-07,"[0.03133528922998593, 0.4139334683181057, 0.24...","[0.17682096462082617, 0.3778087182334166, 0.16...","[0.052657282681602725, 0.1734398226090915, 0.1...","[0.1082503266600593, 0.08673954369361679, 0.47...","[0.0909095214073416, 0.17517057889947232, 0.03...","[0.05230132872750237, 0.11014579327801023, 0.0...","[0.3458586987387225, 0.011801408710086983, 0.0...","[0.08874759884354395, 0.09541444382909235, 0.2...","[0.0746019201987149, 0.14267133731397627, 0.18...","[0.12634027908183787, 0.4655858541430654, 0.25...",...,"[0.1538651862670293, 0.16840158983729536, 0.21...","[0.5663015486162394, 0.0848999752681054, 0.020...","[0.03045904637220715, 0.22646384089471625, 0.5...","[0.16478469756314423, 0.1522

In [ ]:
RPS4=pd.DataFrame()

for k in prob4_final.index: #for everyday of a stock->index
  prob=pd.DataFrame()
  prob1=pd.DataFrame()
  predic=pd.DataFrame()
  if(prob4_final['date'][k]!=prob4_final['date'].tail(1).values): #if it's not for the last day 
    #for historical data of two days 
    prob=rd.loc[(rd.index.date==prob1_final['date'][k])]
    prob1=rd.loc[ (rd.index.date==prob1_final['date'][k+1])]
    prob=create_historical(prob)
    prob1=create_historical(prob1)
    hist_data=prob.append(prob1)
    # print(hist_data)
    predic=prob4_final.loc[prob4_final['date']==prob4_final['date'][k]] #find predictions of this day
    predic=predic.drop('date',axis=1)
    # print(predic)
    data=pd.DataFrame(index=predic.columns)
    data=data.reset_index()
    # print(data)
    data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
    for i in range(len(np.array_split(predic.values[0],46))-1):
      # print(i)
      # print(np.array_split(predic.values[0],46)[i][0])
      data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
    data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
    # print(data)
    #now we have data->predictions and hist_data->historical data so now we can call the RPS
    rps4=RPS_calculation(hist_data = hist_data , submission = data)['details']
    # print(rps) ->rps for every stock for one day 
    # print(rps['RPS'])
    tmp=pd.DataFrame(rps4['RPS'].values).T
    # print(tmp.T)
    RPS4=RPS4.append(tmp,ignore_index=True)
    # print(RPS[RPS.index==k])

    
  else:
    #for the last day
    prob=rd.loc[(rd.index.date==prob1_final['date'][k])]
    prob1=rd.loc[ (rd.index.date==rd.index.date[-1])]
    prob=create_historical(prob)
    prob1=create_historical(prob1)
    hist_data=prob.append(prob1)
    # print(hist_data)
    predic=prob4_final.loc[prob4_final['date']==prob4_final['date'][k]] #find predictions of this day
    predic=predic.drop('date',axis=1)
    # print(predic)
    data=pd.DataFrame(index=predic.columns)
    data=data.reset_index()
    # print(data)
    data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
    for i in range(len(np.array_split(predic.values[0],46))-1):
      # print(i)
      # print(np.array_split(predic.values[0],46)[i][0])
      data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
    data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
    # print(data)
    #now we have data->predictions and hist_data->historical data so now we can call the RPS
    rps4=RPS_calculation(hist_data = hist_data , submission = data)['details']
    # print(rps) ->rps for every stock for one day 
    # print(rps['RPS'])
    tmp=pd.DataFrame(rps4['RPS'].values).T
    # print(tmp.T)
    RPS4=RPS4.append(tmp,ignore_index=True)              
print(RPS4)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == dat

          0         1         2         3         4         5         6   \
0   0.224895  0.295247  0.055784  0.098178  0.388593  0.072539  0.118858   
1   0.132931  0.075694  0.271743  0.115276  0.100215  0.454259  0.125854   
2   0.049886  0.349478  0.135192  0.114299  0.130974  0.039641  0.010178   
3   0.285876  0.084549  0.387078  0.157240  0.289519  0.139158  0.184988   
4   0.071003  0.093759  0.144423  0.131219  0.103607  0.117737  0.377815   
5   0.056997  0.098518  0.075029  0.001768  0.052254  0.211038  0.115573   
6   0.290439  0.346540  0.136435  0.069586  0.111519  0.324322  0.249128   
7   0.247167  0.094970  0.128357  0.061859  0.039212  0.077839  0.140946   
8   0.408896  0.155932  0.166881  0.103763  0.283436  0.211611  0.208846   
9   0.108455  0.139144  0.199502  0.229567  0.192225  0.296091  0.193692   
10  0.187659  0.098846  0.094456  0.464741  0.407258  0.156239  0.302129   
11  0.119616  0.226443  0.269321  0.195459  0.300672  0.499119  0.273560   
12  0.117239

In [ ]:
d4=prob4_final['date']
name4=prob4_final.columns[:-1]
# print(np.split(name.values,len(name)))
# print(d.values)
RPS4['date']=d4.values
RPS4=RPS4.set_index('date')
RPS4.columns=assets
RPS4

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-01-04,0.224895,0.295247,0.055784,0.098178,0.388593,0.072539,0.118858,0.079267,0.147586,0.047680,...,0.198048,0.147953,0.048905,0.416675,0.190525,0.175679,0.223687,0.067671,0.187436,0.222417
2022-01-05,0.132931,0.075694,0.271743,0.115276,0.100215,0.454259,0.125854,0.062002,0.087316,0.032241,...,0.177118,0.141739,0.282430,0.160258,0.059442,0.197665,0.302615,0.112201,0.354911,0.408620
2022-01-06,0.049886,0.349478,0.135192,0.114299,0.130974,0.039641,0.010178,0.193448,0.153325,0.147053,...,0.161202,0.172774,0.430720,0.172016,0.213413,0.007271,0.075972,0.119519,0.251826,0.332832
2022-01-07,0.285876,0.084549,0.387078,0.157240,0.289519,0.139158,0.184988,0.065896,0.388946,0.079656,...,0.098012,0.101404,0.149649,0.107466,0.140463,0.546853,0.111285,0.282397,0.086452,0.106008
2022-01-10,0.071003,0.093759,0.144423,0.131219,0.103607,0.117737,0.377815,0.234363,0.170832,0.111519,...,0.123289,0.183262,0.184064,0.203790,0.123874,0.154180,0.085062,0.032455,0.058793,0.430046
2022-01-11,0.056997,0.098518,0.075029,0.001768,0.052254,0.211038,0.115573,0.115445,0.276322,0.180482,...,0.097798,0.119016,0.283516,0.098298,0.452256,0.249112,0.428429,0.134977,0.306425,0.194618
2022-01-12,0.290439,0.346540,0.136435,0.069586,0.111519,0.324322,0.249128,0.122440,0.073737,0.122289,...,0.142284,0.414541,0.341065,0.156965,0.147324,0.261897,0.079263,0.114143,0.250169,0.159218
2022-01-13,0.247167,0.094970,0.128357,0.061859,0.039212,0.077839,0.140946,0.008241,0.136980,0.318548,...,0.330658,0.336437,0.306869,0.281796,0.150863,0.118244,0.460752,0.090172,0.291628,0.496769
2022-01-14,0.408896,0.155932,0.166881,0.103763,0.283436,0.211611,0.208846,0.019320,0.151535,0.106592,...,0.100154,0.208823,0.278328,0.144737,0.335823,0.096684,0.185981,0.290276,0.354062,0.349473


#####Final RPS calculation for GB

In [ ]:
print(RPS4.mean(axis=1).mean())

0.19257575108656763


#####Model XGB

In [ ]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
model5=XGBClassifier()

In [ ]:
prob5=dict()

for j in assets:
    tmp5=[]
    df=stock[j]
    df1=df.dropna()
    X= df1.iloc[:,5:14]
    # print(X) #it has all the days
    y= df1['Rank']
    X_train= X.iloc[:-60] #only the days of one year until 3/01
    # print(X_train)
    Y_train= y.iloc[:-60]
    # print(Y_train)
    model5.fit(X_train,Y_train)
    #now each day we will use the models to predict 
    for i in range(59,0,-1): #from 4/01 and later -->predictions
      X_validation=X.iloc[-(i+1):-i] 
      # print(X_validation)
      Y_validation= y.iloc[-(i+1):-i]
      print(Y_validation.index) 
      probability5= model5.predict_proba(X_validation)
      print(probability5[0])
      tmp5.append(probability5[0])
    X_validation=X.tail(1) 
    # print(X_validation)
    Y_validation= y.tail(1)
    print(Y_validation.index) 
    probability5= model5.predict_proba(X_validation)
    print(probability5[0])
    tmp5.append(probability5[0])
    prob5[j]=tmp5

Streaming output truncated to the last 5000 lines.
DatetimeIndex(['2022-02-02'], dtype='datetime64[ns]', name='Date', freq=None)
[0.06239115 0.21725272 0.13844514 0.48062578 0.10128517]
DatetimeIndex(['2022-02-03'], dtype='datetime64[ns]', name='Date', freq=None)
[0.0697765  0.20830919 0.04626075 0.424339   0.25131458]
DatetimeIndex(['2022-02-04'], dtype='datetime64[ns]', name='Date', freq=None)
[0.09419537 0.4088754  0.0393592  0.05968793 0.3978821 ]
DatetimeIndex(['2022-02-07'], dtype='datetime64[ns]', name='Date', freq=None)
[0.0642195  0.3048434  0.07208689 0.42421314 0.13463709]
DatetimeIndex(['2022-02-08'], dtype='datetime64[ns]', name='Date', freq=None)
[0.27397645 0.30178553 0.09113355 0.2602247  0.07287979]
DatetimeIndex(['2022-02-09'], dtype='datetime64[ns]', name='Date', freq=None)
[0.18740503 0.37337255 0.10519844 0.13308768 0.20093627]
DatetimeIndex(['2022-02-10'], dtype='datetime64[ns]', name='Date', freq=None)
[0.18040225 0.2965329  0.0569291  0.303171   0.16296472]
Date

In [ ]:
prob5['date']=X.iloc[-60:].index
prob5_final=pd.DataFrame.from_dict(prob5)
prob5_final.set_index('date')

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-01-04,"[0.060994297, 0.29048336, 0.27892724, 0.290525...","[0.15318607, 0.17274795, 0.36937395, 0.2359709...","[0.05433181, 0.08938382, 0.11012323, 0.3833217...","[0.23348136, 0.29840115, 0.32547373, 0.0307551...","[0.20140238, 0.110259585, 0.07414953, 0.010269...","[0.033825003, 0.12251927, 0.4420414, 0.1203494...","[0.24071541, 0.03031917, 0.34429818, 0.2717789...","[0.13253175, 0.22636233, 0.2847763, 0.27505118...","[0.34955373, 0.078728214, 0.10912744, 0.065320...","[0.06703996, 0.30069, 0.15592234, 0.30878472, ...",...,"[0.30606115, 0.054095082, 0.04596884, 0.350911...","[0.3394762, 0.108277015, 0.03392457, 0.0618256...","[0.06648823, 0.2944936, 0.47148228, 0.05013262...","[0.26911968, 0.12242887, 0.12382359, 0.2355704...","[0.1792642, 0.42768097, 0.07298666, 0.23799692...","[0.10191368, 0.5830371, 0.2539841, 0.016657038...","[0.16199103, 0.21601461, 0.08889307, 0.0675941...","[0.23318474, 0.32038057, 0.26291257, 0.1178223...","[0.29782304, 0.09826289, 0.21047392, 0.2926616...","[0.33836418, 0.109089345, 0.07580684, 0.305316..."
2022-01-05,"[0.049134996, 0.1291472, 0.18988, 0.3196562, 0...","[0.2699303, 0.2518303, 0.30044854, 0.16031529,...","[0.019585302, 0.043819614, 0.23185566, 0.26580...","[0.36716142, 0.089708336, 0.035209943, 0.46164...","[0.20283337, 0.36331722, 0.0345883, 0.01322942...","[0.028461706, 0.6260449, 0.007474798, 0.169841...","[0.37868795, 0.02378111, 0.3350395, 0.06136093...","[0.2550507, 0.042129036, 0.37485853, 0.2236570...","[0.28025758, 0.1658252, 0.23741402, 0.07020454...","[0.073248155, 0.5325287, 0.035444148, 0.348117...",...,"[0.017133305, 0.11379415, 0.35153, 0.09448264,...","[0.21130237, 0.08953008, 0.06784407, 0.0683686...","[0.02290843, 0.13710222, 0.65532714, 0.0235883...","[0.11490443, 0.10848423, 0.28312433, 0.3484931...","[0.19222917, 0.36203665, 0.060042877, 0.293697...","[0.21143702, 0.3119435, 0.36744037, 0.03589284...","[0.037682995, 0.2354663, 0.008867619, 0.116856...","[0.071484, 0.10967866, 0.6505107, 0.14734517, ...","[0.044183772, 0.015827341, 0.815932, 0.1064915...","[0.20270072, 0.50700825, 0.021106584, 0.124192..."
2022-01-06,"[0.034661777, 0.08870259, 0.3550527, 0.2304065...","[0.08785222, 0.5269478, 0.060779974, 0.2738835...","[0.05702155, 0.07405346, 0.17583112, 0.1208190...","[0.093255244, 0.45983368, 0.10750196, 0.271832...","[0.47025812, 0.22104166, 0.01919284, 0.0321086...","[0.039352752, 0.30022633, 0.03998698, 0.402816...","[0.37888914, 0.048236787, 0.4723802, 0.0473504...","[0.094375625, 0.19696513, 0.1389874, 0.3844041...","[0.048216477, 0.044213764, 0.09034764, 0.74441...","[0.111568674, 0.39570335, 0.32540905, 0.132140...",...,"[0.13965304, 0.19145666, 0.23849775, 0.1285264...","[0.36074147, 0.12722977, 0.0107962, 0.11192579...","[0.077407934, 0.65137583, 0.06261181, 0.050065...","[0.15544406, 0.43259588, 0.29398707, 0.0402259...","[0.18487608, 0.23054135, 0.10829373, 0.1062637...","[0.09976859, 0.5010978, 0.24754496, 0.0448466,...","[0.41314033, 0.20455846, 0.18504103, 0.0239888...","[0.11998007, 0.16910522, 0.4166816, 0.28145918...","[0.13328971, 0.18039316, 0.37474683, 0.1380989...","[0.034726933, 0.8211011, 0.0052975556, 0.01445..."
2022-01-07,"[0.04527307, 0.22241853, 0.45236418, 0.2217375...","[0.12390691, 0.29274765, 0.4016955, 0.14564492...","[0.068490714, 0.052889396, 0.17583136, 0.56956...","[0.04952819, 0.070900634, 0.6092272, 0.1913883...","[0.11928783, 0.3031978, 0.0412659, 0.12387571,...","[0.039321594, 0.15884499, 0.011387275, 0.69763...","[0.48089603, 0.04592371, 0.14843604, 0.1121298...","[0.05403863, 0.110436425, 0.3250357, 0.4524977...","[0.028558848, 0.2673692, 0.053964693, 0.363994...","[0.14226489, 0.26425338, 0.36071968, 0.1184697...",...,"[0.04613083, 0.18478222, 0.24794412, 0.0652479...","[0.47430593, 0.10132018, 0.024523566, 0.051583...","[0.028944645, 0.34677958, 0.38625348, 0.088040...","[0.071682036, 0.33439517, 0.

In [ ]:
RPS5=pd.DataFrame()

for k in prob5_final.index: #for everyday of a stock->index
  prob=pd.DataFrame()
  prob1=pd.DataFrame()
  predic=pd.DataFrame()
  if(prob5_final['date'][k]!=prob5_final['date'].tail(1).values): #if it's not for the last day 
    #for historical data of two days 
    prob=rd.loc[(rd.index.date==prob1_final['date'][k])]
    prob1=rd.loc[ (rd.index.date==prob1_final['date'][k+1])]
    prob=create_historical(prob)
    prob1=create_historical(prob1)
    hist_data=prob.append(prob1)
    # print(hist_data)
    predic=prob5_final.loc[prob5_final['date']==prob5_final['date'][k]] #find predictions of this day
    predic=predic.drop('date',axis=1)
    # print(predic)
    data=pd.DataFrame(index=predic.columns)
    data=data.reset_index()
    # print(data)
    data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
    for i in range(len(np.array_split(predic.values[0],46))-1):
      # print(i)
      # print(np.array_split(predic.values[0],46)[i][0])
      data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
    data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
    # print(data)
    #now we have data->predictions and hist_data->historical data so now we can call the RPS
    rps5=RPS_calculation(hist_data = hist_data , submission = data)['details']
    # print(rps) ->rps for every stock for one day 
    # print(rps['RPS'])
    tmp=pd.DataFrame(rps5['RPS'].values).T
    # print(tmp.T)
    RPS5=RPS5.append(tmp,ignore_index=True)
    # print(RPS[RPS.index==k])

    
  else:
    #for the last day
    prob=rd.loc[(rd.index.date==prob1_final['date'][k])]
    prob1=rd.loc[ (rd.index.date==rd.index.date[-1])]
    prob=create_historical(prob)
    prob1=create_historical(prob1)
    hist_data=prob.append(prob1)
    # print(hist_data)
    predic=prob5_final.loc[prob5_final['date']==prob5_final['date'][k]] #find predictions of this day
    predic=predic.drop('date',axis=1)
    # print(predic)
    data=pd.DataFrame(index=predic.columns)
    data=data.reset_index()
    # print(data)
    data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
    for i in range(len(np.array_split(predic.values[0],46))-1):
      # print(i)
      # print(np.array_split(predic.values[0],46)[i][0])
      data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
    data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
    # print(data)
    #now we have data->predictions and hist_data->historical data so now we can call the RPS
    rps5=RPS_calculation(hist_data = hist_data , submission = data)['details']
    # print(rps) ->rps for every stock for one day 
    # print(rps['RPS'])
    tmp=pd.DataFrame(rps5['RPS'].values).T
    # print(tmp.T)
    RPS5=RPS5.append(tmp,ignore_index=True)              
print(RPS5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  # Remove the CWD from sys.path while we load stuff.


AttributeError: ignored